In [29]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
# from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.utils.data import DataLoader
from torchmetrics.text import BLEUScore
from transformers.optimization import AdamW
from tqdm.notebook import tqdm_notebook
import torchmetrics
from torch.utils.tensorboard import SummaryWriter

import torch.nn.functional as F
import numpy as np

In [30]:
writer = SummaryWriter('experiments_results')

In [31]:
torch.cuda.is_available()


True

In [64]:
device = torch.device('cuda:0')

In [33]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/xlm-roberta-large-squad2"


# nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
# QA_input = {
#     'question': 'Beyons neçə yaşında LaTavia Robertsonla tanış oldu?',
#     'context': """Səkkiz yaşında, Beyonce və uşaqlıq dostu Kelly Rowland, bütün qızlardan ibarət əyləncə qrupunun dinləmələrində LaTavia Roberson ilə tanış oldular. Onlar Girl's Tyme kimi daha üç qızla bir qrupa yerləşdirildilər və Hyustondakı istedad şousunda rap etdi və rəqs etdilər. Qrupu gördükdən sonra R&B prodüseri Arne Frager onları Şimali Kaliforniya studiyasına gətirdi və o zamanlar milli televiziyada ən böyük istedad şousu olan Star Search-ə yerləşdirdi. Girl's Tyme qalib gələ bilmədi və Beyonce daha sonra ifa etdikləri mahnının yaxşı olmadığını söylədi. 1995-ci ildə Beyonsun atası qrupu idarə etmək üçün işindən istefa verdi. Bu addım Beyons ailəsinin gəlirini yarıbayarı azaltdı və valideynləri ayrı mənzillərə köçmək məcburiyyətində qaldılar. Mathew orijinal heyəti dördə kəsdi və qrup digər qurulmuş R&B qız qrupları üçün açılış aktı kimi çıxış etməyə davam etdi. Qızlar səsyazma şirkətlərindən əvvəl dinləmələrdən keçdilər və nəhayət, Elektra Records ilə müqavilə imzaladılar, ilk səsyazmaları üzərində işləmək üçün qısa müddətə Atlanta Records-a köçdülər, ancaq şirkət tərəfindən kəsildi. Bu, ailəni daha da gərginləşdirdi və Beyonsun valideynləri ayrıldı. 5 oktyabr 1995-ci ildə Dwayne Wiggins-in Grass Roots Entertainment qrupu ilə müqavilə imzaladı. 1996-cı ildə qızlar Sony Music ilə razılaşma əsasında debüt albomlarını yazmağa başladılar, Knowles ailəsi yenidən birləşdi və qısa müddət sonra qrup Columbia Records ilə müqavilə bağladı.	
# """
# }
# res = nlp(QA_input)


model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
device

device(type='cuda', index=0)

In [35]:
# model = model.to(device)
# tokenizer = tokenizer().to(device)

In [36]:
# device = torch.device('mps')
print(device)
# model = model.to(device)
# tokenizer = tokenizer.to(device)

mps


In [37]:
model

XLMRobertaForQuestionAnswering(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_feature

In [38]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

questions = [
    """"19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?"""]

text = r"""Qalan qrup üzvləri 2000-ci ildə çəkilmiş Charlie's Angels filminin soundtrackində görünən "Müstəqil Qadınlar Part I" mahnısını yazdılar. Ardıcıl on bir həftə ərzində ABŞ-ın Billboard Hot 100 cədvəlində birinci yeri tutaraq, onların ən yaxşı chart olan sinqlı oldu. 2001-ci ilin əvvəlində, Destiny's Child üçüncü albomunu tamamlayarkən, Beyonce amerikalı aktyor Mekhi Phifer ilə birlikdə MTV-nin televiziya üçün hazırlanmış "Carmen: A Hip Hopera" filmində böyük rol aldı. Filadelfiyada cərəyan edən film fransız bəstəkarı Georges Bizetin 19-cu əsrdə “Karmen” operasının müasir şərhidir. 2001-ci ilin may ayında üçüncü "Survivor" albomu çıxanda Luckett və Roberson mahnıların onlara yönəldiyini iddia edərək məhkəməyə müraciət etdilər. Albom ilk həftədə 663.000 nüsxə satışı ilə ABŞ Billboard 200-də bir nömrədə debüt etdi. Albom digər bir nömrəli hitləri, "Bootylicious" və baş treki olan "Survivor"u doğurdu, sonuncusu qrupa Duo və ya Vokallı Qrup tərəfindən Ən Yaxşı R&B Performansı üçün Qremmi Mükafatı qazandırdı. 2001-ci ilin oktyabrında 8 Days of Christmas adlı bayram albomunu buraxdıqdan sonra qrup solo karyeralarını davam etdirmək üçün fasilə elan etdi."""



for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    print(inputs)
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(inputs.keys())
    res = model(**inputs)
    print(res)
   
    answer_start = torch.argmax(
        res['start_logits'])
      # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(res['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score
    start = F.softmax(res['start_logits'], dim=1)
    end = F.softmax(res['end_logits'], dim=1)

    start_index = torch.argmax((start), dim=1)
    end_index = torch.argmax((end), dim=1)
    
    outer = np.matmul(np.expand_dims(start.detach().numpy(), -1), np.expand_dims(end.detach().numpy(), 1))
    max_answer_len = 512
    candidates = np.tril(np.triu(outer), max_answer_len - 1)
    idx_sorts = [np.argmax(candidates[i].flatten()) for i in range(len(candidates))]
    scores = [candidates[[i], start_index[i], end_index[i]][0] for i in range(len(candidates))]
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
    print(f"Score: {scores}\n")

{'input_ids': tensor([[     0,     44,   2947,      9,   1010, 123324,   1477,     44,  32759,
           1055,     58,   6264,  13339,  33379,  27333,  10782,    876, 142412,
           1153,   1057,  20735,  63361,     32,      2,      2, 176734,     19,
          51540,  79538,  58970,    318,  11607,  61831,  29919,  71305,     25,
              7,  26902,      7,   1346,   3266, 217148,   8049,  38522,   7879,
             44, 231993, 112582,    320,  16995,     87,     58, 136575,  13339,
         131137,    320,      5,  43794,  22668,    141,     98,    263,  96839,
          32191,  19154,      9,   1110, 211441,   9903,    805,      6,  19414,
          32113,  49031,   8049,  33842,  42375,  12956,  23126,      4,  21475,
           9625,  19795, 116287,   1425,    880,    864,   1304,   7037,      5,
           6789,      9,    318,  14034,  28847,   8049,      4,   5581,  57278,
             25,      7, 123829,  80821,  22053,    561,    539,  25669,  23987,
         14265

In [39]:
from datasets import load_dataset

In [40]:
dataset = load_dataset('vrashad/squad_azerbaijan')

In [41]:
import pandas as pd

In [42]:
df_train = pd.DataFrame(dataset['train'],columns=dataset['train'].features)
df_test = pd.DataFrame(dataset['test'],columns=dataset['test'].features)

In [43]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [44]:
df_test.head()

,id,title,context,question,answer_text,answer_start,is_impossible
0,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
1,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
2,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
3,56ddde6b9a695914005b9628,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandiya hansı ölkədə yerləşir?,Fransa,159.0,False
4,56ddde6b9a695914005b9629,Normans,Normanlar (Normand: Nourmands; fransızca: Norm...,Normandlar nə vaxt Normandiyada idilər?,10 və 11-ci əsrlər,94.0,False


In [45]:
def get_index_of_word_in_sentence(context,answer,answer_start):
    if answer is not None:
        word = answer.split()[0].lower()
        try:
            answer_start_new = context.lower().index(word)
            return {'answer_start':[int(answer_start_new)],'text':[answer]}
        except:
            return {'answer_start':[int(answer_start)],'text':[answer]}
    else:
        return {'answer_start':[],'text':[]}

df_train['answers']= df_train.apply(lambda x: get_index_of_word_in_sentence(x.context,x.answer_text,x.answer_start),axis=1)
df_train.drop(columns=['answer_text','answer_start'],inplace=True)

df_test['answers']= df_test.apply(lambda x: get_index_of_word_in_sentence(x.context,x.answer_text,x.answer_start),axis=1)
df_test.drop(columns=['answer_text','answer_start'],inplace=True)
# df['answers']= df.apply(lambda x: print(x.question),axis=1)

In [46]:
df_train

,id,title,context,question,is_impossible,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyonce nə vaxt populyarlaşmağa başladı?,False,"{'answer_start': [247], 'text': ['1990-cı illə..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons böyüyərkən hansı sahələrdə yarışırdı?,False,"{'answer_start': [203], 'text': ['mahnı oxumaq..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyonce nə vaxt Destiny's Child-dən ayrılaraq ...,False,"{'answer_start': [534], 'text': ['2003']}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons hansı şəhərdə və əyalətdə böyüdü?,False,"{'answer_start': [166], 'text': ['Hyuston, Tex..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons hansı onillikdə məşhurlaşdı?,False,"{'answer_start': [247], 'text': ['1990-cı illə..."
...,...,...,...,...,...,...
130046,5735d259012e2f140011a09d,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Katmandu ilk dəfə ABŞ-ın hansı ştatında beynəl...,False,"{'answer_start': [173], 'text': ['Oreqon']}"
130047,5735d259012e2f140011a09e,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Yanqon əvvəllər nə kimi tanınırdı?,False,"{'answer_start': [292], 'text': ['Ranqun']}"
130048,5735d259012e2f140011a09f,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Katmandunun Belarusun hansı şəhəri ilə əlaqəsi...,False,"{'answer_start': [342], 'text': ['Minsk']}"
130049,5735d259012e2f140011a0a0,Kathmandu,"Katmandu Böyükşəhər Şəhəri (KMC), beynəlxalq ə...",Katmandu ilk beynəlxalq əlaqələri neçənci ildə...,False,"{'answer_start': [153], 'text': ['1975']}"


In [47]:
from datasets import Dataset, DatasetDict

In [48]:
datasets_n = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
    })

In [49]:
datasets_n

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'is_impossible', 'answers', '__index_level_0__'],
        num_rows: 86820
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'is_impossible', 'answers', '__index_level_0__'],
        num_rows: 20302
    })
})

In [50]:
batch_size = 1
max_length = 512 
doc_stride = 128
pad_on_right = tokenizer.padding_side == "right"

In [51]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
 


    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [52]:
df_train[:10000]

,id,title,context,question,is_impossible,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyonce nə vaxt populyarlaşmağa başladı?,False,"{'answer_start': [247], 'text': ['1990-cı illə..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons böyüyərkən hansı sahələrdə yarışırdı?,False,"{'answer_start': [203], 'text': ['mahnı oxumaq..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyonce nə vaxt Destiny's Child-dən ayrılaraq ...,False,"{'answer_start': [534], 'text': ['2003']}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons hansı şəhərdə və əyalətdə böyüdü?,False,"{'answer_start': [166], 'text': ['Hyuston, Tex..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyonce Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyons hansı onillikdə məşhurlaşdı?,False,"{'answer_start': [247], 'text': ['1990-cı illə..."
...,...,...,...,...,...,...
13027,56df81465ca0a614008f9ba0,Oklahoma_City,"""Oklahoman"" Oklahoma City-nin əsas gündəlik qə...",Oklahomanın veb-saytının adı nədir?,False,"{'answer_start': [92], 'text': ['NewsOK.com']}"
13028,56df81465ca0a614008f9ba1,Oklahoma_City,"""Oklahoman"" Oklahoma City-nin əsas gündəlik qə...",Oklahoma Cities həftəlik xəbər nədir?,False,"{'answer_start': [1], 'text': ['Oklahoma Qəzet..."
13029,56df81465ca0a614008f9ba2,Oklahoma_City,"""Oklahoman"" Oklahoma City-nin əsas gündəlik qə...",Oklahoma Cities gündəlik iş qəzeti nədir?,False,"{'answer_start': [338], 'text': ['Jurnal Rekor..."
13030,56df81b556340a1900b29c8e,Oklahoma_City,Yerli olaraq şəhərin etnik mozaikasına uyğun g...,Qara Salnamələr Qərargahı hansı tərəfdədir?,False,"{'answer_start': [156], 'text': ['Şərq tərəf']}"


In [53]:
train = df_train[:1000].sample(frac=1, random_state=42)


df_train = train[:-64].reset_index(drop=True)
df_valid = train[-64:].reset_index(drop=True)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [54]:
train_dataset[0]


{'id': '56bfc563a10cfb14005512cc',
 'title': 'Beyoncé',
 'context': 'Musiqi jurnalisti Ture, Dangerously in Love filminin yayımlanmasından bəri onun "krossover seks simvoluna çevrildiyini" yazaraq, Beyons geniş miqyaslı cinsi cazibədar biri kimi təsvir edilmişdir. Səhnədənkənar Beyonce deyir ki, o, seksual geyinməyi sevsə də, onun səhnə geyimi "tamamilə səhnə üçündür". Onun əyriliklərinə və bu terminin cəlbediciliyinə görə 2000-ci illərdə media tez-tez "Bootylicious" terminindən (booty və dadlı sözlərinin portmantosu) Beyonceni təsvir etmək üçün istifadə edirdi, bu termin Destiny\'s Child\'ın eyni adlı sinqlı ilə populyarlaşır. 2006-cı ildə Oksford İngilis dili lüğətinə əlavə edildi.',
 'question': 'Bu termin lüğətə nə vaxt əlavə edilib?',
 'is_impossible': False,
 'answers': {'answer_start': [565], 'text': ['2006']}}

In [55]:
train_dataset[0]['context'][20:]

're, Dangerously in Love filminin yayımlanmasından bəri onun "krossover seks simvoluna çevrildiyini" yazaraq, Beyons geniş miqyaslı cinsi cazibədar biri kimi təsvir edilmişdir. Səhnədənkənar Beyonce deyir ki, o, seksual geyinməyi sevsə də, onun səhnə geyimi "tamamilə səhnə üçündür". Onun əyriliklərinə və bu terminin cəlbediciliyinə görə 2000-ci illərdə media tez-tez "Bootylicious" terminindən (booty və dadlı sözlərinin portmantosu) Beyonceni təsvir etmək üçün istifadə edirdi, bu termin Destiny\'s Child\'ın eyni adlı sinqlı ilə populyarlaşır. 2006-cı ildə Oksford İngilis dili lüğətinə əlavə edildi.'

In [56]:
tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
valid_dataset = Dataset.from_pandas(df_valid)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/936 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [57]:
tokenized_valid_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 64
})

In [58]:
device

device(type='mps')

In [59]:
EPOCHS = 1

In [60]:
train_dataloader = torch.utils.data.DataLoader(dataset=tokenized_train_ds, batch_size=1)

In [61]:
for i in tokenized_train_ds:
    print(i)

{'input_ids': [0, 667, 18614, 6, 5374, 5467, 52773, 9366, 24281, 28012, 20152, 32, 2, 2, 216858, 58276, 14, 4252, 13, 4, 2290, 1505, 79850, 23, 11954, 1346, 3266, 96557, 1414, 137126, 48816, 416, 9826, 44, 16417, 10798, 814, 6530, 10777, 23232, 76, 65268, 98503, 58, 20735, 23126, 4, 41505, 5245, 21379, 6, 228636, 132580, 377, 708, 25955, 1506, 7479, 5238, 201347, 108516, 5, 49493, 127, 19, 136619, 56407, 147, 233495, 55893, 200, 4, 36, 4, 24265, 700, 9298, 85954, 20838, 12749, 2216, 4, 9826, 194129, 176456, 14, 44, 2537, 1191, 3855, 194129, 2385, 20688, 740, 33890, 17125, 63273, 597, 20206, 530, 373, 46576, 19, 65870, 93824, 135199, 4154, 58970, 318, 72928, 2450, 21405, 9, 19544, 44, 12647, 72057, 150, 60744, 58, 18614, 26375, 15, 80765, 53, 530, 17901, 1304, 112190, 694, 11469, 669, 1952, 34, 16, 233495, 93, 201347, 11187, 2385, 11466, 145014, 4, 373, 18614, 5581, 57278, 25, 7, 123829, 25, 1110, 45683, 30081, 880, 864, 1304, 1594, 33408, 6828, 108560, 5, 3295, 9, 5924, 11607, 9972, 7,

In [65]:
tokenized_train_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 950
})

In [66]:
model = model.to(device)

In [67]:
import torch.nn.functional as F

In [68]:
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output

tensor(1.9356, grad_fn=<NllLossBackward0>)

In [69]:
target

tensor([3, 1, 4])

In [70]:
input

tensor([[-0.3493, -0.2337,  1.0214, -1.2717,  0.1906],
        [ 0.1806,  2.2834,  1.9851, -0.7776,  0.5319],
        [ 0.3349, -0.0787, -1.4503, -0.7611, -0.8044]], requires_grad=True)

In [71]:
model = model.to(device)

In [72]:
len(tokenized_train_ds)

950

In [73]:
def fine_tuning():
    print('Start fine-tuning')  
    optimizer = AdamW(model.parameters(), lr=1e-4)
    model.train()
    for i in range(EPOCHS):
        losses = 0
        z = 0
        # batch_iterator = tqdm_notebook(train_dataloader,total=len(list(train_dataloader)) ,desc=f"Processing Epoch {i:02d}")
        for d in tokenized_train_ds:
                data = {'input_ids':torch.as_tensor([d['input_ids']]).to(device),'attention_mask':torch.as_tensor([d['attention_mask']]).to(device)}
                
                output = model(**data)
                
                start_pred = output['start_logits'].to(torch.float).to(device)
                end_pred = output['end_logits'].to(torch.float ).to(device)
                
                start_label, end_label = torch.LongTensor([d['start_positions']]).to(device), torch.LongTensor([d['end_positions']]).to(device)
                
                loss = F.cross_entropy(start_pred, start_label) + F.cross_entropy(end_pred, end_label)
        
                torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
                # loss = output.loss
                loss.backward()
                
                optimizer.step()
                optimizer.zero_grad()
                
                losses += loss.item()
                i += 1
                print(i)
        losses = losses / len(list(train_dataloader))
        print(f'Epoch: {i}, Losses: {losses}')
        # evaluate()

In [74]:
fine_tuning()

Start fine-tuning


c:\Users\Admin\Desktop\Squad\SQuAD_Azerbaijan\env\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [80]:
test_dataset = Dataset.from_pandas(df_valid)
tokenized_test_ds = test_dataset.map(prepare_train_features, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [81]:
df_valid

,id,title,context,question,is_impossible,answers
0,56beca973aeaaa14008c9479,Beyoncé,"Onun debüt sinqlı ""Crazy in Love"" VH1-in ""2000...","Crazy in Love, tarixin ən çox satılan sinqllar...",False,"{'answer_start': [267], 'text': ['8 milyon']}"
1,56d4d5ef2ccc5a1400d83286,Beyoncé,2015-ci ildə Beyonce ONE Kampaniyasının imza t...,Beyonce 2015-ci ildə kimin üçün məktub imzaladı?,False,"{'answer_start': [44], 'text': ['BİR Kampaniya']}"
2,56d4c6b02ccc5a1400d83224,Beyoncé,2013-cü ilin yanvarında Destiny's Child əvvəlk...,Prezident Obamanın ikinci andiçmə mərasimində ...,False,"{'answer_start': [259], 'text': ['amerikan mil..."
3,56bf95eaa10cfb1400551195,Beyoncé,Onun 2006-cı ildə ilk aktyor rolu Stiv Martinl...,Beyons ilk konsert turunu necə adlandırdı?,False,"{'answer_start': [489], 'text': ['Beyonce Təcr..."
4,56cf5bfcaab44d1400b89115,Frédéric_Chopin,Friderikin atası Nikolas Şopen 1787-ci ildə on...,Şopenin böyük bacısının adı nədir?,False,"{'answer_start': [548], 'text': ['Lüdvika']}"
...,...,...,...,...,...,...
59,56d4831f2ccc5a1400d83155,Beyoncé,Qalan qrup üzvləri 2000-ci ildə çəkilmiş Charl...,Beyonce 2001-ci ildə Mekhi Phifer ilə hansı fi...,False,"{'answer_start': [424], 'text': ['Carmen: Hip ..."
60,56bfa24ea10cfb14005511e7,Beyoncé,2013-cü ilin yanvarında Destiny's Child əvvəlk...,Yarım saat şousu dəqiqədə neçə Tweet aldı?,False,"{'answer_start': [418], 'text': ['Dəqiqədə 268..."
61,56cbdea66d243a140015edae,Frédéric_Chopin,"Ola bilsin ki, Fryderyk anasından bir az piano...",Frederik neçə yaşında ictimai konsertlər vermə...,False,"{'answer_start': [365], 'text': ['7']}"
62,56bfb8dca10cfb1400551279,Beyoncé,"O, Destiny's Child ilə yazılan mahnıların əksə...",Beyonsun ilk musiqisi hansı mövzuda idi?,False,"{'answer_start': [166], 'text': ['qadın səlahi..."


In [82]:
def compute_f1(a_gold, a_pred): 
    gold_toks = get_tokens (a_gold) 
    pred_toks = get_tokens (a_pred)
    common  = collections. Counter (gold_toks) & collections. Counter (pred_toks) 
    num_same = sum (common. values())
    if len(gold_toks) == or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise 
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision =  1.0 * num_same / len(pred_toks)
    recall =  1.0 *  num_same / len(gold_toks)
    f1 = (2 * precision *  recall) / (precision + recall)
    return f1

SyntaxError: invalid syntax (1413718088.py, line 6)

In [83]:
questions = [
    """"19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?"""]

text = r"""Qalan qrup üzvləri 2000-ci ildə çəkilmiş Charlie's Angels filminin soundtrackində görünən "Müstəqil Qadınlar Part I" mahnısını yazdılar. Ardıcıl on bir həftə ərzində ABŞ-ın Billboard Hot 100 cədvəlində birinci yeri tutaraq, onların ən yaxşı chart olan sinqlı oldu. 2001-ci ilin əvvəlində, Destiny's Child üçüncü albomunu tamamlayarkən, Beyonce amerikalı aktyor Mekhi Phifer ilə birlikdə MTV-nin televiziya üçün hazırlanmış "Carmen: A Hip Hopera" filmində böyük rol aldı. Filadelfiyada cərəyan edən film fransız bəstəkarı Georges Bizetin 19-cu əsrdə “Karmen” operasının müasir şərhidir. 2001-ci ilin may ayında üçüncü "Survivor" albomu çıxanda Luckett və Roberson mahnıların onlara yönəldiyini iddia edərək məhkəməyə müraciət etdilər. Albom ilk həftədə 663.000 nüsxə satışı ilə ABŞ Billboard 200-də bir nömrədə debüt etdi. Albom digər bir nömrəli hitləri, "Bootylicious" və baş treki olan "Survivor"u doğurdu, sonuncusu qrupa Duo və ya Vokallı Qrup tərəfindən Ən Yaxşı R&B Performansı üçün Qremmi Mükafatı qazandırdı. 2001-ci ilin oktyabrında 8 Days of Christmas adlı bayram albomunu buraxdıqdan sonra qrup solo karyeralarını davam etdirmək üçün fasilə elan etdi."""



for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    print(inputs)
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(inputs.keys())
    res = model(**inputs.to(device))
    print(res)
   
    answer_start = torch.argmax(
        res['start_logits'])
      # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(res['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score
    start = F.softmax(res['start_logits'], dim=1)
    end = F.softmax(res['end_logits'], dim=1)

    start_index = torch.argmax((start), dim=1)
    end_index = torch.argmax((end), dim=1)
    
    outer = np.matmul(np.expand_dims(start.detach().numpy(), -1), np.expand_dims(end.detach().numpy(), 1))
    max_answer_len = 512
    candidates = np.tril(np.triu(outer), max_answer_len - 1)
    idx_sorts = [np.argmax(candidates[i].flatten()) for i in range(len(candidates))]
    scores = [candidates[[i], start_index[i], end_index[i]][0] for i in range(len(candidates))]
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
    print(f"Score: {scores}\n")

{'input_ids': tensor([[     0,     44,   2947,      9,   1010, 123324,   1477,     44,  32759,
           1055,     58,   6264,  13339,  33379,  27333,  10782,    876, 142412,
           1153,   1057,  20735,  63361,     32,      2,      2, 176734,     19,
          51540,  79538,  58970,    318,  11607,  61831,  29919,  71305,     25,
              7,  26902,      7,   1346,   3266, 217148,   8049,  38522,   7879,
             44, 231993, 112582,    320,  16995,     87,     58, 136575,  13339,
         131137,    320,      5,  43794,  22668,    141,     98,    263,  96839,
          32191,  19154,      9,   1110, 211441,   9903,    805,      6,  19414,
          32113,  49031,   8049,  33842,  42375,  12956,  23126,      4,  21475,
           9625,  19795, 116287,   1425,    880,    864,   1304,   7037,      5,
           6789,      9,    318,  14034,  28847,   8049,      4,   5581,  57278,
             25,      7, 123829,  80821,  22053,    561,    539,  25669,  23987,
         14265

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [84]:
model = model.to(device)

In [85]:
for i in df_train[:20].iterrows():
    answer = i[1]['answers']['text'][0]
    # print(i[1]['context'][0])
    inputs = tokenizer.encode_plus(i[1]['question'], i[1]['context'], add_special_tokens=True, return_tensors="pt").to(device)
    input_ids = torch.as_tensor(inputs["input_ids"].tolist()[0])
    res = model(**inputs)
    
    answer_start = torch.argmax(
        res['start_logits'])
      # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(res['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score
    start = F.softmax(res['start_logits'], dim=1)
    end = F.softmax(res['end_logits'], dim=1)

    start_index = torch.argmax((start), dim=1)
    end_index = torch.argmax((end), dim=1)
    
    outer = np.matmul(np.expand_dims(start.cpu().detach().numpy(), -1), np.expand_dims(end.cpu().detach().numpy(), 1))
    max_answer_len = 512
    candidates = np.tril(np.triu(outer), max_answer_len - 1)
    idx_sorts = [np.argmax(candidates[i].flatten()) for i in range(len(candidates))]
    scores = [candidates[[i], start_index[i], end_index[i]][0] for i in range(len(candidates))]
    
    answer_pred = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"answer_pred: {answer_pred}\n")
    print(f"Answer: {answer}\n")
    print(f"Score: {scores}\n")
    

Question: "19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?
answer_pred: 

Answer: 2006

Score: [0.0]

Question: "19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?
answer_pred: ons və anası Tina Brooklyn Phoenix House-da Beyons Kosmetologiya Mərkəzini açıb, kişilər və qadınlar üçün yeddi aylıq kosmetologiya kursu təklif ediblər. 2011-ci ilin aprelində Beyonce ABŞ-ın birinci xanımı Mişel Obama və Milli Yayımçılar Təhsil Fondu ilə birləşərək, "Get Me Bodied" adlı sinqlının yenidən işlənməsi ilə uşaq piylənməsinə qarşı kampaniyanı gücləndirməyə kömək etdi. Üsamə bin Ladenin ölümündən sonra Beyons Nyu-York Polisi və Yanğın Dulları və Uşaqlara Yardım Fonduna vəsait toplamaq üçün xeyriyyə sinqlı kimi Lee Greenwood-un "God Bless the USA" mahnı

Answer: İndi Haiti üçün Ümid: Qlobal Fayda

Score: [8.4083564e-05]

Question: "19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?
answer_pred: . . 5 mart 2010-cu ildə Beyons və anası Tina Brooklyn Phoenix

In [ ]:
def evaluate():
    print('Start evaluate')  
    evaluate
    for d in tokenized_test_ds:
            data = {'input_ids':torch.as_tensor([d['input_ids']]).to(device),'attention_mask':torch.as_tensor([d['attention_mask']]).to(device)}
            
            output = model(**data)
            
            start_pred = output['start_logits'].to(torch.float).to(device)
            end_pred = output['end_logits'].to(torch.float ).to(device)
            
            start_label, end_label = torch.LongTensor([d['start_positions']]).to(device), torch.LongTensor([d['end_positions']]).to(device)
            
            loss = F.cross_entropy(start_pred, start_label) + F.cross_entropy(end_pred, end_label)
    
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
            # loss = output.loss
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            
            losses += loss.item()
            i += 1
            print(i)
    losses = losses / len(list(train_dataloader))
    print(f'Epoch: {i}, Losses: {losses}')

In [17]:

for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    print(inputs)
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(inputs.keys())
    res = model(**inputs)
    print(res)
   
    answer_start = torch.argmax(
        res['start_logits'])
      # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(res['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score
    start = F.softmax(res['start_logits'], dim=1)
    end = F.softmax(res['end_logits'], dim=1)

    start_index = torch.argmax((start), dim=1)
    end_index = torch.argmax((end), dim=1)
    
    outer = np.matmul(np.expand_dims(start.detach().numpy(), -1), np.expand_dims(end.detach().numpy(), 1))
    max_answer_len = 512
    candidates = np.tril(np.triu(outer), max_answer_len - 1)
    idx_sorts = [np.argmax(candidates[i].flatten()) for i in range(len(candidates))]
    scores = [candidates[[i], start_index[i], end_index[i]][0] for i in range(len(candidates))]
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

{'input_ids': tensor([[     0,     44,   2947,      9,   1010, 123324,   1477,     44,  32759,
           1055,     58,   6264,  13339,  33379,  27333,  10782,    876, 142412,
           1153,   1057,  20735,  63361,     32,      2,      2, 176734,     19,
          51540,  79538,  58970,    318,  11607,  61831,  29919,  71305,     25,
              7,  26902,      7,   1346,   3266, 217148,   8049,  38522,   7879,
             44, 231993, 112582,    320,  16995,     87,     58, 136575,  13339,
         131137,    320,      5,  43794,  22668,    141,     98,    263,  96839,
          32191,  19154,      9,   1110, 211441,   9903,    805,      6,  19414,
          32113,  49031,   8049,  33842,  42375,  12956,  23126,      4,  21475,
           9625,  19795, 116287,   1425,    880,    864,   1304,   7037,      5,
           6789,      9,    318,  14034,  28847,   8049,      4,   5581,  57278,
             25,      7, 123829,  80821,  22053,    561,    539,  25669,  23987,
         14265